In [1]:
%matplotlib inline
import math
import os

import matplotlib.pyplot as plt
import numpy as np
from numpy import arange
import pandas as pd

from scipy import stats #importa apenas as funções de estatísticas da biblioteca SciPy.

from IPython.display import display  # Para ter melhor print.

In [2]:
dados = pd.read_csv('MICRODADOS_ENEM_2023.csv', encoding='latin-1', sep=';')


In [3]:
# Conferindo se todas as linhas são de 2023
dados['NU_ANO'].value_counts()

2023    3933955
Name: NU_ANO, dtype: int64

In [4]:
# Vendo todas as colunas do DataFrame
list(dados)

['NU_INSCRICAO',
 'NU_ANO',
 'TP_FAIXA_ETARIA',
 'TP_SEXO',
 'TP_ESTADO_CIVIL',
 'TP_COR_RACA',
 'TP_NACIONALIDADE',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA',
 'TP_ENSINO',
 'IN_TREINEIRO',
 'CO_MUNICIPIO_ESC',
 'NO_MUNICIPIO_ESC',
 'CO_UF_ESC',
 'SG_UF_ESC',
 'TP_DEPENDENCIA_ADM_ESC',
 'TP_LOCALIZACAO_ESC',
 'TP_SIT_FUNC_ESC',
 'CO_MUNICIPIO_PROVA',
 'NO_MUNICIPIO_PROVA',
 'CO_UF_PROVA',
 'SG_UF_PROVA',
 'TP_PRESENCA_CN',
 'TP_PRESENCA_CH',
 'TP_PRESENCA_LC',
 'TP_PRESENCA_MT',
 'CO_PROVA_CN',
 'CO_PROVA_CH',
 'CO_PROVA_LC',
 'CO_PROVA_MT',
 'NU_NOTA_CN',
 'NU_NOTA_CH',
 'NU_NOTA_LC',
 'NU_NOTA_MT',
 'TX_RESPOSTAS_CN',
 'TX_RESPOSTAS_CH',
 'TX_RESPOSTAS_LC',
 'TX_RESPOSTAS_MT',
 'TP_LINGUA',
 'TX_GABARITO_CN',
 'TX_GABARITO_CH',
 'TX_GABARITO_LC',
 'TX_GABARITO_MT',
 'TP_STATUS_REDACAO',
 'NU_NOTA_COMP1',
 'NU_NOTA_COMP2',
 'NU_NOTA_COMP3',
 'NU_NOTA_COMP4',
 'NU_NOTA_COMP5',
 'NU_NOTA_REDACAO',
 'Q001',
 'Q002',
 'Q003',
 'Q004',
 'Q005',
 'Q006',
 'Q007',
 'Q008',
 'Q009

In [5]:
# Selecionando as colunas do nosso interesse
colunas = ['NU_INSCRICAO', 
           'TP_FAIXA_ETARIA',
            'TP_SEXO',
            'TP_ESTADO_CIVIL', 
            'TP_COR_RACA', 
            'TP_ESCOLA',  
            'IN_TREINEIRO', 
            'TP_PRESENCA_CN', 
            'TP_PRESENCA_CH', 
            'TP_PRESENCA_LC', 
            'TP_PRESENCA_MT', 
            'NU_NOTA_CN',
            'NU_NOTA_CH', 
            'NU_NOTA_LC', 
            'NU_NOTA_MT', 
            'TP_STATUS_REDACAO', 
            'NU_NOTA_REDACAO']

dados_small = pd.read_csv('MICRODADOS_ENEM_2023.csv', encoding='latin-1', sep=';', usecols=colunas)
dados_small

,NU_INSCRICAO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ESCOLA,IN_TREINEIRO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TP_STATUS_REDACAO,NU_NOTA_REDACAO
0,210059085136,14,M,2,1,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,210059527735,12,M,2,1,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,210061103945,6,F,1,1,1,0,1,1,1,1,502.0,498.9,475.6,363.2,1.0,700.0
3,210060214087,2,F,1,3,2,0,1,1,1,1,459.0,508.5,507.2,466.7,1.0,880.0
4,210059980948,3,F,1,3,2,0,1,1,1,1,402.5,379.2,446.9,338.3,1.0,560.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3933950,210061959676,12,M,1,1,1,0,1,1,1,1,566.2,605.6,613.7,547.3,1.0,540.0
3933951,210061950911,1,F,1,1,1,1,1,1,1,1,377.2,535.6,610.6,644.4,1.0,640.0
3933952,210061965966,3,F,1,3,2,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3933953,210061932304,2,M,1,1,3,0,1,1,1,1,515.7,539.0,536.0,569.0,1.0,600.0


In [6]:
# Conferindo se é possível que um aluno tenha sido dado com presente em uma prova e ausente em outra que acontecem no mesmo dia
pd.crosstab(dados_small.TP_PRESENCA_CH, dados_small.TP_PRESENCA_LC)

TP_PRESENCA_LC,0,1,2
TP_PRESENCA_CH,,,
0,1106714,0,0
1,0,2822643,0
2,0,0,4598


In [7]:
# Selecionando apenas os alunos que foram no primeiro dia

dados_small_presentes_dia_1 = dados_small.drop(dados_small[(dados_small.TP_PRESENCA_CH != 1)]. index )
dados_small_presentes_dia_1

,NU_INSCRICAO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ESCOLA,IN_TREINEIRO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TP_STATUS_REDACAO,NU_NOTA_REDACAO
2,210061103945,6,F,1,1,1,0,1,1,1,1,502.0,498.9,475.6,363.2,1.0,700.0
3,210060214087,2,F,1,3,2,0,1,1,1,1,459.0,508.5,507.2,466.7,1.0,880.0
4,210059980948,3,F,1,3,2,0,1,1,1,1,402.5,379.2,446.9,338.3,1.0,560.0
9,210060801601,11,M,1,1,1,0,1,1,1,1,564.7,630.3,610.4,680.2,1.0,600.0
10,210059085130,8,M,1,3,1,0,1,1,1,1,644.9,620.2,626.9,736.3,1.0,860.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3933946,210061965964,12,F,1,1,1,0,1,1,1,1,568.6,605.2,598.2,496.8,1.0,500.0
3933948,210061959674,3,M,1,1,2,0,1,1,1,1,476.2,542.9,545.2,530.5,1.0,600.0
3933950,210061959676,12,M,1,1,1,0,1,1,1,1,566.2,605.6,613.7,547.3,1.0,540.0
3933951,210061950911,1,F,1,1,1,1,1,1,1,1,377.2,535.6,610.6,644.4,1.0,640.0


#### Formatando a base de Dados

In [8]:
# Vendo os tipos de dados disponíveis
dados_small_presentes_dia_1.dtypes

NU_INSCRICAO           int64
TP_FAIXA_ETARIA        int64
TP_SEXO               object
TP_ESTADO_CIVIL        int64
TP_COR_RACA            int64
TP_ESCOLA              int64
IN_TREINEIRO           int64
TP_PRESENCA_CN         int64
TP_PRESENCA_CH         int64
TP_PRESENCA_LC         int64
TP_PRESENCA_MT         int64
NU_NOTA_CN           float64
NU_NOTA_CH           float64
NU_NOTA_LC           float64
NU_NOTA_MT           float64
TP_STATUS_REDACAO    float64
NU_NOTA_REDACAO      float64
dtype: object

In [9]:
# Adequando as colunas para qualitativas
qualitativas = ['TP_FAIXA_ETARIA', 
                'TP_SEXO',
                'TP_ESTADO_CIVIL',
                'TP_COR_RACA',
                'TP_ESCOLA',
                'IN_TREINEIRO',
                'TP_PRESENCA_CN',
                'TP_PRESENCA_CH',
                'TP_PRESENCA_LC',
                'TP_PRESENCA_MT',
                'TP_STATUS_REDACAO']

for coluna in qualitativas:
    dados_small_presentes_dia_1[coluna] = dados_small_presentes_dia_1[coluna].astype('category')

dados_small_presentes_dia_1.dtypes

NU_INSCRICAO            int64
TP_FAIXA_ETARIA      category
TP_SEXO              category
TP_ESTADO_CIVIL      category
TP_COR_RACA          category
TP_ESCOLA            category
IN_TREINEIRO         category
TP_PRESENCA_CN       category
TP_PRESENCA_CH       category
TP_PRESENCA_LC       category
TP_PRESENCA_MT       category
NU_NOTA_CN            float64
NU_NOTA_CH            float64
NU_NOTA_LC            float64
NU_NOTA_MT            float64
TP_STATUS_REDACAO    category
NU_NOTA_REDACAO       float64
dtype: object

In [10]:
# Renomeado os rótulos das variáveis qualitativas

dados_small_presentes_dia_1['TP_FAIXA_ETARIA'] = dados_small_presentes_dia_1['TP_FAIXA_ETARIA'].cat.rename_categories({1:'Menor de 17 anos', 
                                                                                                                       2:'17 anos',
                                                                                                                       3:'18 anos',
                                                                                                                       4:'19 anos',
                                                                                                                       5:'20 anos', 
                                                                                                                       6:'21 anos',
                                                                                                                       7: '22 anos',
                                                                                                                       8:'23 anos',
                                                                                                                       9:'24 anos',
                                                                                                                       10:'25 anos',
                                                                                                                       11:'Entre 26 e 30 anos',
                                                                                                                       12:'Entre 31 e 40 anos',
                                                                                                                       13:'Entre 36 e 40 anos',
                                                                                                                       14:'Entre 41 e 45 anos',
                                                                                                                       15:'Entre 46 e 50 anos',
                                                                                                                       16:'Entre 51 e 55 anos',
                                                                                                                       17:'Entre 56 e 60 anos',
                                                                                                                       18:'Entre 61 e 65 anos',
                                                                                                                       19:'Entre 66 e 70 anos',
                                                                                                                       20:'Mais de 70 anos'})

dados_small_presentes_dia_1['TP_SEXO'] = dados_small_presentes_dia_1['TP_SEXO'].cat.rename_categories({'M':'Masculino',
                                                                                                       'F':'Feminino'})

dados_small_presentes_dia_1['TP_ESTADO_CIVIL'] = dados_small_presentes_dia_1['TP_ESTADO_CIVIL'].cat.rename_categories({0:'Não informado',
                                                                                                                       1:'Solteiro(a)',
                                                                                                                       2:'Casado(a)/Mora com companheiro(a)',
                                                                                                                       3:'Divorciado(a)/Desquitado(a)/Separado(a)',
                                                                                                                       4:'Viúvo(a)'})

dados_small_presentes_dia_1['TP_COR_RACA'] = dados_small_presentes_dia_1['TP_COR_RACA'].cat.rename_categories({ 0:'Não declarado',
                                                                                                                1:'Branca',
                                                                                                                2:'Preta',
                                                                                                                3:'Parda',
                                                                                                                4:'Amarela',
                                                                                                                5:'Indígena', 
                                                                                                                6:'Não dispõe da informação'})

dados_small_presentes_dia_1['TP_ESCOLA'] = dados_small_presentes_dia_1['TP_ESCOLA'].cat.rename_categories({1:'Não Respondeu',
                                                                                                            2:'Pública',
                                                                                                            3:'Privada'})

dados_small_presentes_dia_1['IN_TREINEIRO'] = dados_small_presentes_dia_1['IN_TREINEIRO'].cat.rename_categories({1:'Sim',
                                                                                                                 0:'Não'})

dados_small_presentes_dia_1['TP_PRESENCA_CN'] = dados_small_presentes_dia_1['TP_PRESENCA_CN'].cat.rename_categories({0:'Faltou',
                                                                                                                     1:'Presente na prova',
                                                                                                                     2:'Eliminado'})

dados_small_presentes_dia_1['TP_PRESENCA_CH'] = dados_small_presentes_dia_1['TP_PRESENCA_CH'].cat.rename_categories({1:'Presente na prova'})

dados_small_presentes_dia_1['TP_PRESENCA_LC'] = dados_small_presentes_dia_1['TP_PRESENCA_LC'].cat.rename_categories({1:'Presente na prova'})

dados_small_presentes_dia_1['TP_PRESENCA_MT'] = dados_small_presentes_dia_1['TP_PRESENCA_MT'].cat.rename_categories({0:'Faltou',
                                                                                                                     1:'Presente na prova',
                                                                                                                     2:'Eliminado'})

dados_small_presentes_dia_1['TP_STATUS_REDACAO'] = dados_small_presentes_dia_1['TP_STATUS_REDACAO'].cat.rename_categories({1:'Sem problemas',
                                                                                                                            2:'Anulada',
                                                                                                                            3:'Cópia Texto Motivador',
                                                                                                                            4:'Em Branco',
                                                                                                                            6:'Fuga ao tema',
                                                                                                                            7:'Não atendimento ao tipo textual',
                                                                                                                            8:'Texto insuficiente',
                                                                                                                            9:'Parte desconectada'})



In [11]:
dados_small_presentes_dia_1.head()

,NU_INSCRICAO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ESCOLA,IN_TREINEIRO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TP_STATUS_REDACAO,NU_NOTA_REDACAO
2,210061103945,21 anos,Feminino,Solteiro(a),Branca,Não Respondeu,Não,Presente na prova,Presente na prova,Presente na prova,Presente na prova,502.0,498.9,475.6,363.2,Sem problemas,700.0
3,210060214087,17 anos,Feminino,Solteiro(a),Parda,Pública,Não,Presente na prova,Presente na prova,Presente na prova,Presente na prova,459.0,508.5,507.2,466.7,Sem problemas,880.0
4,210059980948,18 anos,Feminino,Solteiro(a),Parda,Pública,Não,Presente na prova,Presente na prova,Presente na prova,Presente na prova,402.5,379.2,446.9,338.3,Sem problemas,560.0
9,210060801601,Entre 26 e 30 anos,Masculino,Solteiro(a),Branca,Não Respondeu,Não,Presente na prova,Presente na prova,Presente na prova,Presente na prova,564.7,630.3,610.4,680.2,Sem problemas,600.0
10,210059085130,23 anos,Masculino,Solteiro(a),Parda,Não Respondeu,Não,Presente na prova,Presente na prova,Presente na prova,Presente na prova,644.9,620.2,626.9,736.3,Sem problemas,860.0


In [12]:
# Conferindo os rótulos que cada coluna de presença possui
print(dados_small_presentes_dia_1['TP_PRESENCA_CH'].value_counts())
print(dados_small_presentes_dia_1['TP_PRESENCA_LC'].value_counts())
print(dados_small_presentes_dia_1['TP_PRESENCA_MT'].value_counts())
print(dados_small_presentes_dia_1['TP_PRESENCA_CN'].value_counts())

Presente na prova    2822643
Name: TP_PRESENCA_CH, dtype: int64
Presente na prova    2822643
Name: TP_PRESENCA_LC, dtype: int64
Presente na prova    2678264
Faltou                142219
Eliminado               2160
Name: TP_PRESENCA_MT, dtype: int64
Presente na prova    2678264
Faltou                142219
Eliminado               2160
Name: TP_PRESENCA_CN, dtype: int64


In [13]:
# Criando a coluna "PRESENCA_PRIMEIRO_DIA" e "PRESENCA_SEGUNDO_DIA"

presente_dia_1 = dados_small_presentes_dia_1['TP_PRESENCA_CH'] == 'Presente na prova'
presente_dia_2 = dados_small_presentes_dia_1['TP_PRESENCA_MT'] == 'Presente na prova'
eliminado_dia_2 = dados_small_presentes_dia_1['TP_PRESENCA_MT'] == 'Eliminado'
faltou_dia_2 = dados_small_presentes_dia_1['TP_PRESENCA_MT'] == 'Faltou'

# Criando a coluna "PRESENCA_PRIMEIRO_DIA"
dados_small_presentes_dia_1.loc[presente_dia_1, 'PRESENCA_PRIMEIRO_DIA'] = 'Presente'

# Criando a coluna "PRESENCA_SEGUNDO_DIA"
# Os eliminados no segundo dia serão considerados presentes
dados_small_presentes_dia_1.loc[presente_dia_2 | eliminado_dia_2, 'PRESENCA_SEGUNDO_DIA'] = 'Presente'
dados_small_presentes_dia_1.loc[faltou_dia_2, 'PRESENCA_SEGUNDO_DIA'] = 'Faltou'

In [14]:
dados_small_presentes_dia_1.head(10)

,NU_INSCRICAO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ESCOLA,IN_TREINEIRO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TP_STATUS_REDACAO,NU_NOTA_REDACAO,PRESENCA_PRIMEIRO_DIA,PRESENCA_SEGUNDO_DIA
2,210061103945,21 anos,Feminino,Solteiro(a),Branca,Não Respondeu,Não,Presente na prova,Presente na prova,Presente na prova,Presente na prova,502.0,498.9,475.6,363.2,Sem problemas,700.0,Presente,Presente
3,210060214087,17 anos,Feminino,Solteiro(a),Parda,Pública,Não,Presente na prova,Presente na prova,Presente na prova,Presente na prova,459.0,508.5,507.2,466.7,Sem problemas,880.0,Presente,Presente
4,210059980948,18 anos,Feminino,Solteiro(a),Parda,Pública,Não,Presente na prova,Presente na prova,Presente na prova,Presente na prova,402.5,379.2,446.9,338.3,Sem problemas,560.0,Presente,Presente
9,210060801601,Entre 26 e 30 anos,Masculino,Solteiro(a),Branca,Não Respondeu,Não,Presente na prova,Presente na prova,Presente na prova,Presente na prova,564.7,630.3,610.4,680.2,Sem problemas,600.0,Presente,Presente
10,210059085130,23 anos,Masculino,Solteiro(a),Parda,Não Respondeu,Não,Presente na prova,Presente na prova,Presente na prova,Presente na prova,644.9,620.2,626.9,736.3,Sem problemas,860.0,Presente,Presente
11,210060801595,18 anos,Masculino,Solteiro(a),Parda,Pública,Não,Presente na prova,Presente na prova,Presente na prova,Presente na prova,608.2,667.6,607.9,691.9,Sem problemas,780.0,Presente,Presente
13,210060214081,18 anos,Feminino,Solteiro(a),Branca,Não Respondeu,Não,Presente na prova,Presente na prova,Presente na prova,Presente na prova,626.5,604.9,614.3,548.8,Sem problemas,800.0,Presente,Presente
14,210059388087,Entre 26 e 30 anos,Feminino,Solteiro(a),Preta,Não Respondeu,Não,Presente na prova,Presente na prova,Presente na prova,Presente na prova,547.2,623.3,593.7,404.6,Sem problemas,560.0,Presente,Presente
15,210059980942,22 anos,Feminino,Solteiro(a),Parda,Não Respondeu,Não,Presente na prova,Presente na prova,Presente na prova,Presente na prova,510.8,313.4,388.5,415.6,Sem problemas,560.0,Presente,Presente
16,210058061534,19 anos,Feminino,Solteiro(a),Branca,Pública,Não,Presente na prova,Presente na prova,Presente na prova,Presente na prova,515.7,553.1,544.4,437.0,Sem problemas,880.0,Presente,Presente


In [15]:
# Data Frame pronto, iremos transformá-lo em um arquivo csv mais leve para usá-lo definitivamente
dados_small_presentes_dia_1.to_csv('MICRODADOS_REDUZIDO',sep=';')